In [3]:
import pandas as pd

import requests
import json

from datetime import datetime, timedelta

from pprint import  pprint



In [9]:
lang =     "es"
category = "intercambios"
widget =   "enlace-baleares"
query =    "start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=day"


In [5]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [10]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url= endpoint, headers= headers)
data = response.json()
pprint(data)


{'data': {'attributes': {'description': None,
                         'last-update': '2019-04-15T15:32:01.000+02:00',
                         'title': 'Enlace Península-Baleares'},
          'id': 'int9',
          'meta': {'cache-control': {'cache': 'MISS'}},
          'type': 'Enlace Península-Baleares'},
 'included': [{'attributes': {'color': '#28b3da',
                              'composite': False,
                              'description': None,
                              'last-update': '2019-04-15T15:32:01.000+02:00',
                              'magnitude': None,
                              'title': 'Salidas',
                              'type': 'export',
                              'values': [{'datetime': '2019-01-01T00:00:00.000+01:00',
                                          'percentage': 0.5,
                                          'value': -3181.766},
                                         {'datetime': '2019-01-02T00:00:00.000+01:00',
               